# 03. Transfer Learning (ResNet50)

## Introduction
This notebook implements Transfer Learning using a pre-trained ResNet50 model.
It is fully self-contained.

## Setup

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

tf.random.set_seed(42)
np.random.seed(42)

2025-11-24 10:10:49.468598: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-24 10:10:49.636615: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/mathieu/Works/Projet-PIC/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/mathieu/Works/Projet-PIC/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major

## 1. Data Loading
Using the same strategy as the baseline: training on the multi-class data from the test folder for demonstration.

In [3]:

def load_data(processed_dir, target_category=None, img_size=(256, 256), batch_size=32):
    """
    Loads ALL data from processed_dir (augmented) and splits into Train/Val/Test (80/10/10).
    """
    logger.info(f"Loading augmented data for category: {target_category}...")
    
    if not os.path.exists(processed_dir):
        raise ValueError(f"Processed directory {processed_dir} not found! Please run augmentation first.")
        
    # 1. Load All Augmented Data
    all_files = []
    logger.info(f"Scanning processed data in {processed_dir}...")
    # Structure: processed_dir/{category}_{defect}/{filename}.png
    search_path = os.path.join(processed_dir, "*", "*.png")
    
    for filepath in glob.glob(search_path):
        folder_name = os.path.basename(os.path.dirname(filepath))
        # folder_name is like 'bottle_good' or 'bottle_broken_large'
        if target_category and not folder_name.startswith(target_category):
            continue
        
        all_files.append({
            'filepath': filepath,
            'label_str': folder_name
        })
            
    df = pd.DataFrame(all_files)
    if df.empty:
        raise ValueError("No data found in processed directory!")
        
    # 2. Create Label Mapping
    all_labels = sorted(list(df['label_str'].unique()))
    label_to_idx = {label: i for i, label in enumerate(all_labels)}
    df['label'] = df['label_str'].map(label_to_idx)
    
    # 3. Split Data (80% Train, 10% Val, 10% Test)
    # First split: Train (80%) vs Temp (20%)
    train_df, temp_df = train_test_split(df, train_size=0.8, random_state=42, stratify=df['label'])
    
    # Second split: Val (50% of 20% = 10%) vs Test (50% of 20% = 10%)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])
    
    logger.info(f"Train samples: {len(train_df)}, Val samples: {len(val_df)}, Test samples: {len(test_df)}")
    logger.info(f"Classes: {len(all_labels)}")
    
    # 4. Create Datasets with ResNet Preprocessing
    def process_path(filepath, label):
        img = tf.io.read_file(filepath)
        img = tf.io.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, img_size)
        # ResNet specific preprocessing
        img = preprocess_input(img)
        return img, label

    def create_ds(df, shuffle=False):
        ds = tf.data.Dataset.from_tensor_slices((df['filepath'].values, df['label'].values))
        ds = ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return ds

    train_ds = create_ds(train_df, shuffle=True)
    val_ds = create_ds(val_df)
    test_ds = create_ds(test_df)
    
    return train_ds, val_ds, test_ds, len(all_labels), all_labels

# --- Configuration ---
IMG_SIZE = (256, 256)
BATCH_SIZE = 32
PROCESSED_DIR = "../data/processed/augmented"
TARGET_CATEGORY = 'bottle' # Train only on this category

# --- Execution ---
train_ds, val_ds, test_ds, num_classes, class_names = load_data(PROCESSED_DIR, TARGET_CATEGORY, IMG_SIZE, BATCH_SIZE)
print("Data loaded successfully.")


Loading and splitting data for capsule...
Classes: ['capsule_crack', 'capsule_faulty_imprint', 'capsule_good', 'capsule_poke', 'capsule_scratch', 'capsule_squeeze']
Number of classes: 6
Datasets created and preprocessed.


## 2. Model Definition
We load ResNet50 (ImageNet weights), freeze the base, and add a custom classification head.

In [ ]:
def create_resnet_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze base model
    base_model.trainable = False
    
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs, name="resnet50_transfer")
    return model

model = create_resnet_model(IMG_SIZE + (3,), num_classes)
model.summary()

## 3. Training
Training the top layers.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
)

## 4. Fine-tuning (Optional)
Unfreezing the last few layers of ResNet for better performance.

In [ ]:
base_model = model.layers[1]
base_model.trainable = True

# Freeze all except last 20 layers
for layer in base_model.layers[:-20]:
    layer.trainable = False
    
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # Lower LR
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
)

In [ ]:
# Advanced Evaluation

print("Evaluating model with advanced metrics...")
results = evaluate_model(model, test_ds)

print("\nEvaluation Results:")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

## 5. Export Model
We save the trained model for later use.

In [ ]:
# Create models directory if it doesn't exist
models_dir = "../models"
os.makedirs(models_dir, exist_ok=True)

# Save the model
model_name = "resnet50_transfer"
category_name = TARGET_CATEGORY if 'TARGET_CATEGORY' in locals() else 'all_categories'
save_path = os.path.join(models_dir, f"{model_name}_{category_name}.keras")
model.save(save_path)
print(f"Model saved to {save_path}")